## PCA 

Принцип работы метода хорошо описан здесь: https://habr.com/ru/post/304214/

Количество "групп" в данных при каждом запуске PCA равно параметру n_components. Собственное значение матрицы ковариации, соответствующее каждой компоненте, можно считать показателем важности этой группы в наших данны - чем больше это значение, тем больше разброса в данных мы потеряем, если уберем эту группу. Сохранение variance данных - основное ограничение PCA. 

Каждому собственному значению соответствует собственный вектор. Каждый его элемент указывает на то, как коррелированы принадлежность объекта к этой группе и значения признака. Порядок признаков неизменен.

## Просто запустите этот ноутбук с помощью "Restart and Run All"

In [1]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ipywidgets import interact
import seaborn as sns
%matplotlib inline

In [2]:
def plot_pca(pca, eigenvalues):
    tot = sum(eigenvalues)
    var_exp = [(i / tot)*100 for i in sorted(eigenvalues, reverse=True)]
    cum_var_exp = np.cumsum(var_exp)
    with plt.style.context('seaborn-whitegrid'):
        plt.figure(figsize=(6, 4))

        plt.bar(range(pca.n_components_), var_exp, alpha=0.5, align='center',
                label='individual explained variance')
        plt.step(range(pca.n_components_), cum_var_exp, where='mid',
                 label='cumulative explained variance')
        plt.ylabel('Explained variance ratio')
        plt.xlabel('Principal components')
        plt.legend(loc='best')
        plt.tight_layout()
        
        plt.show()

def eigen_to_dict(eigenvalue, eigenvector):    
    eigendict = dict(zip(data.columns, eigenvector))
    eigendict['Eigenvalue'] = eigenvalue
    return eigendict

### Число главных компонент можно изменять с помощью "бегунка", в ячейке ниже можно задать значение "all", "except_pic", "pic" - то, какие признаки учитывать в PCA.

In [ ]:
с = 'all'

In [3]:
data = pd.read_csv('../data/all_features_unification.csv', index_col='vk_id')

columns = {
    'all' : None, 
    'except_pic' : data.columns[:5].to_list() + ['positive_text'], 
    'pic' : ['is_nsfw', 'has_smile', 'number_of_faces', 'has_face']
}

cols = columns[с]

if cols:
    data = data[cols].copy(deep=True)

NameError: name 'с' is not defined

In [ ]:
@interact
def show_pca(n=(2, data.shape[-1], 1)):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    pca = PCA(n_components=n)
    transformed = pca.fit_transform(scaled_data)
    n_samples = scaled_data.shape[0]
    # We center the data and compute the sample covariance matrix.
    scaled_data -= np.mean(scaled_data, axis=0)
    cov_matrix = np.dot(scaled_data.T, scaled_data) / n_samples
    eigenvalues = pca.explained_variance_
    plot_pca(pca, eigenvalues)
    df = pd.DataFrame.from_records([eigen_to_dict(eigenvalue, eigenvector) 
                                    for eigenvalue, eigenvector in zip(eigenvalues, pca.components_)], index=range(1, n+1))
    return df